In [7]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import LinAlgError
from scipy.stats import skew, kurtosis
from PIL import Image
import sys, os
import logging
import argparse, copy
import time

import sutils
import steerable_pyramid_mat as steerable
import texture_analysis_g as ta
from scipy import io
from IPython.core.debugger import Pdb
from pandas import DataFrame as DF
import openpyxl
import pandas as pd

ALPHA = 0.8
'''
	Texture Synthesis by Portilla-Simoncelli's algorithm

'''

out_path ="tmp"
#image = Image.open("samples/blanket2-b-p017_original.png").convert("L")
image = Image.open("periodic/CountExamp1.o.jpg").convert("L")
image = image.resize((256,256))
image = np.array(image)

resol_x = image.shape[1]
resol_y = image.shape[0]
num_depth = 4
num_ori = 4
num_neighbor = 7
iter = 200

cmask = np.array([1,1,1,1])

class Exwritter():
    def __init__(self):
        self.wr_count = 0
        DF([0]).to_excel("output.xlsx")
    def w(self,array):
        df = DF(array)
        writer = pd.ExcelWriter('output.xlsx',engine="openpyxl",mode="a")    
        df.to_excel(writer, sheet_name='Sheet_name_{}'.format(self.wr_count))
        writer.close()
        self.wr_count +=1

In [8]:
# cmask (optional): binary column vector (4x1) indicating which sets of
# constraints we want to apply in the synthesis. The four sets are:
#               1) Marginal statistics (mean, var, skew, kurt, range)
#               2) Correlation of subbands (space, orientation, scale)
#               3) Correlation of magnitude responses (sp, or, sc)
#               4) Relative local phase

In [9]:
#image_mat = io.loadmat("synth-test/init_ref.mat")
#image = image_mat["im0"]

# analyse original image
orig_data = ta.TextureAnalysis(image, resol_x, resol_y, num_depth, num_ori, num_neighbor)
orig_data.analyse()

# center index of auto correlation 
Na = orig_data.LR_CA.shape[0]
la = int((Na -1)/2)
p = 1

In [10]:
#orig_data.IM_MAR[4] = 255
#orig_data.IM_MAR[5] = 0

In [11]:
im = io.loadmat("synth-test/init_im.mat")
im = im["im"]

# iteration
prev_im = np.array([])
prev_dst = 0.
wr = Exwritter()

In [12]:
im = np.random.normal(0, 1, resol_x * resol_y).reshape(resol_y, resol_x)
im = im * np.sqrt(orig_data.IM_MAR[1])
im = im + orig_data.IM_MAR[0]

# iteration
prev_im = np.array([])
prev_dst = 0.

In [13]:
for it in range(0, iter):
    print(it)

    # ------------------------------------
    # Create pyramids of each PCA channel
    # steerable pyramid
    _sp = steerable.SteerablePyramid(im, resol_x, resol_y, num_depth, num_ori, '', '', 0)
    _sp.create_pyramids()

    # subtract means from lowpass residuals
    _sp.LR['s'] = _sp.LR['s'].real - np.mean(_sp.LR['s'].real.flatten())
    
    #get lo of LR
    _lsp = steerable.SteerablePyramid(_sp.LR["s"],_sp.LR["s"].shape[1] ,_sp.LR["s"].shape[0], 1, num_ori, '', '', 0)
    _lsp.create_pyramids() 
    im = _lsp.L0["s"].real # rec_im is partially reconstrucated image
    vari = orig_data.LR_CA[la,la]
   
    # ------------------------------------
    # Adjust lowpass residual and get initial image for coarse to fine
    # modify central auto correlation
    if cmask[1]:
        if (vari/orig_data.IM_MAR[1]) > 1e-4:
            im = sutils.mod_acorr(im, orig_data.LR_CA, num_neighbor)
        else:
            im = im * np.sqrt(orig_data.LR_MAR[1] / np.var(im,ddof=1))
            
    # modify skewness of lowpass residual & modify kurtosis of lowpass residual
    if cmask[0]:
        if (vari/orig_data.IM_MAR[1]) > 1e-4  :
            im = sutils.mod_skew(im, orig_data.LR_MAR[2])
            im = sutils.mod_kurt(im, orig_data.LR_MAR[3])
            
    rec_im = im
#    Pdb().set_trace()#############################################################################################

    ## get original statistics of bandpass signals.
    # create parents
    if cmask[2]:
        bnd = copy.deepcopy(_sp.BND)
        _b_m, _, _ = sutils.trans_b(_sp.BND) # return magnitude,real,imag
        for i in range(len(_b_m)):
            for k in range(len(_b_m[i])):
                _b_m[i][k] -= np.mean(_b_m[i][k])
        ## magnitude
        bnd_m = _b_m

    ## maginitude of parent bandpass  (this is 'parent' in textureColorAnalysis.m)
    ## real values of parent bandpass (this is half of 'rparent' in textureColorAnalysis.m)
    ## imaginary values of parent bandpass (this is half of 'rparent' in textureColorAnalysis.m)

    # ------------------------------------
    # Coarse to fine adjustment
    for dp in range(num_depth-1, -1, -1):
        
        if (cmask[2] or cmask[3]):
            bnd_p, bnd_rp, bnd_ip = sutils.get_parent_g(_sp.BND)

        if cmask[2]:
            # combine orientations
            cousins = sutils.cori_b(bnd_m, dp)
            # adjust covariances
            
            if dp < num_depth-1:
                parents = bnd_p[dp]
                _tmp = sutils.adjust_corr2s(cousins, orig_data.CF_COUS[dp], parents, orig_data.CF_CPAR[dp])

            else: # dp = 4 no pair．
                _tmp = sutils.adjust_corr1(cousins, orig_data.CF_COUS[dp])

            if ( np.var(_tmp.flatten().imag ) / np.var( _tmp.flatten().real) ) > 1e-6:
                # non update cousins (bnd_magnitude) 
                print("Non- trivial imaginary part, mag cross Corr lev = {}".format(dp))
            else: 
                cousins = _tmp.real
                bnd_m[dp] = np.array([ci.reshape( bnd_m[dp][0].shape[0],bnd_m[dp][0].shape[1] ) for ci in  cousins.T])
            
  #      Pdb().set_trace()###############################################################################################
        # separate orientations
        #cousins=[K][Y*X] to [K][Y,X] 
        #cousins = sutils.sori_b(cousins, num_ori) # [K][Y,X]にするやつ sqrt使ってるからダメ．
        #bnd_m[dp] = np.array([ci.reshape( bnd_m[dp][0].shape[0],bnd_m[dp][0].shape[1] ) for ci in  cousins.T])

        # adjust central auto corr. and update bandpass.
        bnd_r = []
        for k in range(num_ori):
            # adjust central auto-correlations
            _tmp = sutils.mod_acorr(bnd_m[dp][k], orig_data.BND_MCOR[dp][k], num_neighbor)

            # update BND_N
            bnd_m[dp][k] = _tmp
            _mean = orig_data.BND_MMAR[dp][k][0]
            bnd_m[dp][k] = bnd_m[dp][k] + _mean
            
            _idx = np.where(bnd_m[dp][k] < 0)
            bnd_m[dp][k][_idx] = 0
            
            #_bnd = _sp.BND[dp][k]['s']
            _idx1 = np.where(np.abs(_sp.BND[dp][k]["s"] ) < 10**(-12))
            _idx2 = np.where(np.abs(_sp.BND[dp][k]["s"] ) >= 10**(-12))
            _sp.BND[dp][k]["s"][_idx1] = _sp.BND[dp][k]["s"][_idx1] * bnd_m[dp][k][_idx1]
            _sp.BND[dp][k]["s"][_idx2] = _sp.BND[dp][k]["s"][_idx2] * bnd_m[dp][k][_idx2] / np.abs(_sp.BND[dp][k]["s"][_idx2])
            
            bnd_r.append(_sp.BND[dp][k]["s"].real)
            
  #      Pdb().set_trace()#######################################################################################################
        # combine orientations & make rcousins
        rcousins = sutils.cori_bc(bnd_r, dp)

        # adjust cross-correlation of real values of B and real/imaginary values of parents    
        # when cmask[3] == 1 only use real part co
        
        if (dp < num_depth-1) & cmask[3]: #
        # deform for optimization
            rparents = sutils.cori_rp(bnd_rp, bnd_ip, dp)
            
            #size = _sp.BND[dp][0]["s"].shape
            #rcousins = [ci.reshape( size[0],size[1] ).T.flatten() for ci in  rcousins.T]
            #rcousins = np.array(rcousins).T
            
            _tmp = np.zeros_like(rcousins)
            for k in range(num_ori):
                cou = rcousins[:,k].reshape(-1,1)
                _mean = np.mean(cou.flatten()**2).reshape(1,1)
                Cxy = orig_data.CF_RPAR[dp][k].reshape(1,-1)
                #_tmp = sutils.adjust_corr2(cou, orig_data.CF_RCOU[dp], rparents, orig_data.CF_RPAR[dp][k])
                cou = sutils.adjust_corr2s(cou,_mean , rparents, Cxy)
                _tmp[:,k] = cou.flatten()
                
            #_tmp = [ci.reshape( size[0],size[1] ).T.flatten() for ci in  _tmp.T]
           # _tmp = np.array(_tmp).T
            
        else:
            #_tmp = sutils.adjust_corr1(rcousins, orig_data.CF_RCOU[dp])
            _tmp = rcousins
            
        ''' matlab Code で最下層のみでの更新はない．したほうがいいんじゃね？
        else:
            rcousins = sutils.adjust_corr1(_prev, orig_data.CF_RCOU[dp])
            if np.isnan(rcousins).any():
                rcousins = _prev
        '''

        if ( np.var(_tmp.flatten().imag ) / np.var( _tmp.flatten().real) ) > 1e-6:
            print("Non- trivial imaginary part, CF_RCOU lev = {}".format(dp))
        else:
            rcousins = _tmp
            # [K,Y*X] => [K,Y,X]
            # This sets real part only - signal is now nonanalytic!
            size = _sp.BND[dp][0]["s"].shape
            rcousins = [ci.reshape( size[0],size[1] ) for ci in  rcousins.T]
            rcousins = np.array(rcousins)
            for k in range(num_ori):
                _sp.BND[dp][k]["s"] = rcousins[k]
            
  #      Pdb().set_trace()################################################################################################################
        # Re-create analytic subbands
        dims = np.asarray(_sp.BND[dp][0]["s"].shape,"float")
        ctr = np.ceil((dims + 0.5)/2).astype("int")
        ang = sutils.mkAngle(dims,0,ctr)
        ang[ctr[0]-1,ctr[1]-1] = -np.pi/2
        
        for k in range(num_ori): 
            ang0 = np.pi*(k-1 + 1)/num_ori
            xang = (ang-ang0 + np.pi)%(2*np.pi) - np.pi
            amask = abs( abs(xang) - np.pi/2 ) < 1e-10
            amask = amask + 2* ( abs(xang) < (np.pi/2) )
            amask[ctr[0]-1,ctr[1]-1] = 1
            amask[:,0] = 1
            amask[0,:] = 1        
            amask = np.fft.fftshift(amask)
            _sp.BND[dp][k]["s"] = np.fft.ifft2(amask * np.fft.fft2( _sp.BND[dp][k]["s"] ) )

    #    Pdb().set_trace()#######################################################################################################
        # same size
        _z = np.zeros_like(_sp.BND[dp][0]["s"].real)
        _s = steerable.SteerablePyramid(_z, _z.shape[1], _z.shape[0], 1, num_ori, '', '', 0)#create_filt_only
        _recon = np.zeros_like(_z,dtype=np.complex128)
        
        for k in range(num_ori):
            _recon = _recon + ( (1j)**(num_ori-1) ) * _s.B_FILT[0][k] * np.fft.fftshift( np.fft.fft2( _sp.BND[dp][k]['s'].real ) )

        _recon = _recon * _s.L0_FILT
        _recon = np.fft.ifft2(np.fft.ifftshift(_recon)).real

        # expand image created before and sum up
        _im = rec_im
        _im = sutils.expand(_im, 2).real / 4.
        _im = _im.real + _recon
        
        vari = orig_data.CF_CA[dp][la,la]
        
        #Pdb().set_trace()#########################################################################################################
        # adjust auto-correlation
        if cmask[1]:
            if (vari/orig_data.IM_MAR[1]) > 1e-4:            
                _im = sutils.mod_acorr(_im, orig_data.CF_CA[dp], num_neighbor)
            else:
                _im = im * np.sqrt( vari / np.var(_im.flatten() ,ddof=1) )
            
        _im = _im.real
        
        if cmask[0]:
        # modify skewness and kurtosis
            if (vari/orig_data.IM_MAR[1]) > 1e-4: 
                _im = sutils.mod_skew(_im, orig_data.CF_MAR[dp][2])
                _im = sutils.mod_kurt(_im, orig_data.CF_MAR[dp][3])
            
        rec_im = _im

    # end of coarse to fine
  #      Pdb().set_trace()#################################################################################################333333333333333333333

    # ------------------------------------
    # Adjustment variance in H0 and final adjustment of coarse to fine.
    if cmask[1] or cmask[2] or cmask[3]:
        _tmp = _sp.H0['s'].real
        vHPR = np.mean(_tmp.flatten()**2)
        
        if vHPR > orig_data.H0_PRO:
            _tmp = _tmp * np.sqrt(orig_data.H0_PRO / vHPR)
            _sp.H0["s"] = _tmp

    # recon H0
    _recon = np.fft.fftshift(np.fft.fft2(_sp.H0["s"]))
    _recon = _recon * _s.H0_FILT
    _recon = np.fft.ifft2(np.fft.ifftshift(_recon)).real

    ## this is final data of coarse to fine.
    rec_im = rec_im + _recon

    # adjust skewness and kurtosis to original.
    _mean = np.mean(rec_im)
    _var = np.var(rec_im,ddof=1)
    rec_im = rec_im - _mean
    
    if cmask[0]:
        rec_im = rec_im * np.sqrt(( (1-p)*_var + p*orig_data.IM_MAR[1] )/_var)
        
    rec_im = rec_im + orig_data.IM_MAR[0]
    
    if cmask[0]:
        rec_im = sutils.mod_skew(rec_im, orig_data.IM_MAR[2])
        rec_im = sutils.mod_kurt(rec_im, orig_data.IM_MAR[3])
    
    _idx  = np.where(rec_im > orig_data.IM_MAR[4])
    rec_im[_idx] = orig_data.IM_MAR[4]
    _idx  = np.where(rec_im < orig_data.IM_MAR[5])
    rec_im[_idx] = orig_data.IM_MAR[5]
    im = rec_im
    
 #   Pdb().set_trace()######################################################################################

    # ------------------------------------
    # Save image
    _o_img = Image.fromarray(np.uint8(im)).convert('L')
    _o_img.save(out_path + '/out-n{}-k{}-m{}-{}.png'.format(str(num_depth), str(num_ori), str(num_neighbor), str(it)))

    if it > 0:
            dst = np.sqrt(np.sum((prev_im - im)**2))
            rt = np.sqrt(np.sum((prev_im - im)**2)) / np.sqrt(np.sum(prev_im**2))

            prev_dst = dst
            # acceleration
            im = im + ALPHA * (im - prev_im)
    prev_im = im


0


[sutils.py] 2019-10-28 07:29:23,168 INFO 545 negative eigenvalue
[sutils.py] 2019-10-28 07:29:23,168 INFO 546 [-156.91768768   28.08469212   18.31370527   20.77190642]


Non- trivial imaginary part, mag cross Corr lev = 0
1


[sutils.py] 2019-10-28 07:29:29,269 INFO 545 negative eigenvalue
[sutils.py] 2019-10-28 07:29:29,269 INFO 546 [-19.29313717  44.16173774  20.07253181  21.03596286]


Non- trivial imaginary part, mag cross Corr lev = 0
2


...

Non- trivial imaginary part, mag cross Corr lev = 0
16


[sutils.py] 2019-10-28 07:30:27,615 INFO 545 negative eigenvalue
[sutils.py] 2019-10-28 07:30:27,618 INFO 546 [60.50866357 -1.5660428  21.97887429 20.72615179]


Non- trivial imaginary part, mag cross Corr lev = 0
17


[sutils.py] 2019-10-28 07:30:32,817 INFO 545 negative eigenvalue
[sutils.py] 2019-10-28 07:30:32,817 INFO 546 [61.24925766 -0.22535901 22.16259619 20.75006546]


Non- trivial imaginary part, mag cross Corr lev = 0
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
